Model Building - Disposal %

In [1]:
import pandas as pd
import numpy as np
import joblib
import warnings
warnings.filterwarnings('ignore')

from expected_disposal_model.config import modelling_file_path, model_output_path, prediction_output_path
from expected_disposal_model.modelling_data_contract import ModellingDataContract
from expected_disposal_model.modelling.hyperparameter_tuning import XGBHyperparameterTuner
from expected_disposal_model.modelling.supermodel import SuperXGBClassifier
from expected_disposal_model.modelling.optuna_xgb_param_grid import OptunaXGBParamGrid

pd.options.display.max_rows = 100
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

Model Version

In [2]:
model_version = 2
model_name = 'disposal'
model_file_name = model_name + '_v' + str(model_version)

In [3]:
RESPONSE = ModellingDataContract.RESPONSE

In [4]:
FEATURES = ModellingDataContract.feature_list

In [5]:
MONOTONE_CONSTRAINTS = ModellingDataContract.monotone_constraints

Load Data

In [6]:
modelling_data = pd.read_csv(modelling_file_path)
modelling_data.tail()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Season,action_type,Contested,Mark,Free,From_Centre_Bounce,Kick_Inside50,To_Ball_Up,From_Ball_Up,Rushed_Behind,Contest_Target,Goal,Behind,To_Out_On_Full,From_Out_On_Full,Error,From_Kick_In,end_x,end_y,pitch_start_x,pitch_start_y,pitch_end_x,pitch_end_y,left_right_start_x,left_right_start_y,left_right_end_x,left_right_end_y,start_distance_to_right_goal,end_distance_to_right_goal,Inside50,1,2,3,4,Duration,match_id,chain_number,order,quarter,quarter_seconds,overall_seconds,team,player,contested,mark,outcome_type,type_Kick_a0,type_Handball_a0,outcome_effective_a0,outcome_ineffective_a0,outcome_clanger_a0,type_Kick_outcome_effective_a0,type_Kick_outcome_ineffective_a0,type_Kick_outcome_clanger_a0,type_Handball_outcome_effective_a0,type_Handball_outcome_ineffective_a0,type_Handball_outcome_clanger_a0,quarter_a0,quarter_seconds_a0,overall_seconds_a0,left_right_start_x_a0,left_right_start_y_a0,left_right_end_x_a0,left_right_end_y_a0,dx_a0,dy_a0,movement_a0,contested_a0,mark_a0,type_Kick_a1,type_Handball_a1,outcome_effective_a1,outcome_ineffective_a1,outcome_clanger_a1,type_Kick_outcome_effective_a1,type_Kick_outcome_ineffective_a1,type_Kick_outcome_clanger_a1,type_Handball_outcome_effective_a1,type_Handball_outcome_ineffective_a1,type_Handball_outcome_clanger_a1,quarter_a1,quarter_seconds_a1,overall_seconds_a1,left_right_start_x_a1,left_right_start_y_a1,left_right_end_x_a1,left_right_end_y_a1,dx_a1,dy_a1,movement_a1,contested_a1,mark_a1,type_Kick_a2,type_Handball_a2,outcome_effective_a2,outcome_ineffective_a2,outcome_clanger_a2,type_Kick_outcome_effective_a2,type_Kick_outcome_ineffective_a2,type_Kick_outcome_clanger_a2,type_Handball_outcome_effective_a2,type_Handball_outcome_ineffective_a2,type_Handball_outcome_clanger_a2,quarter_a2,quarter_seconds_a2,overall_seconds_a2,left_right_start_x_a2,left_right_start_y_a2,left_right_end_x_a2,left_right_end_y_a2,dx_a2,dy_a2,movement_a2,contested_a2,mark_a2,team_1,team_2,time_delta1,time_delta2,dx_a01,dy_a01,move_a01,dx_a02,dy_a02,move_a02,goalscore_team,goalscore_opponent,goalscore_diff,Disposal_Response,Disposal_ResponseTrainingSet,Disposal_ResponseTestSet,Disposal_ResponseValidationSet
466211,240,ballUp,endQuarter,1845.0,4,1755,1771.0,Collingwood,Collingwood,Scott Pendlebury,Scott_Pendlebury,Kick,-45.0,-38.0,effective,NaN,NaN,141,160,Collingwood,Brisbane Lions,right,2023F4_Collingwood_BrisbaneLions,2023F4,NaN,2023.0,Kick,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-29.0,-34.0,-45.0,-38.0,-29.0,-34.0,-45.0,-38.0,-29.0,-34.0,130.648383,114.179683,NaN,2025.0,2134.0,1823.0,1805.0,7753.0,2023F4_Collingwood_BrisbaneLions,240,1845.0,4,1771.0,7753.0,Collingwood,Scott Pendlebury,NaN,NaN,effective,True,False,True,False,False,True,False,False,False,False,False,4,1771.0,7753.0,-45.0,-38.0,-29.0,-34.0,16.0,4.0,16.492423,NaN,NaN,True,False,True,False,False,True,False,False,False,False,False,4,1768.0,7750.0,-23.0,-35.0,-45.0,-38.0,-22.0,-3.0,22.203603,NaN,NaN,False,True,False,True,False,False,False,False,False,True,False,4,1736.0,7718.0,-27.0,-33.0,-23.0,-35.0,4.0,-2.0,4.472136,NaN,NaN,True,True,-3.0,-35.0,0.0,0.0,0.0,22.0,3.0,22.203603,0,0,0,1,True,False,False
466212,240,ballUp,endQuarter,1847.0,4,1755,1779.0,Collingwood,Collingwood,Tom Mitchell,Tom_Mitchell,Kick,-29.0,-34.0,effective,NaN,NaN,141,160,Collingwood,Brisbane Lions,right,2023F4_Collingwood_BrisbaneLions,2023F4,NaN,2023.0,Kick,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-49.0,-48.0,-29.0,-34.0,-49.0,-48.0,-29.0,-34.0,-49.0,-48.0,114.179683,137.640837,NaN,2025.0,2134.0,1823.0,1805.0,7761.0,2023F4_Collingwood_BrisbaneLions,240,1847.0,4,1779.0,7761.0,Collingwood,Tom Mitchell,NaN,NaN,effective,True,False,True,False,False,True,False,Fals

In [7]:
training_data = modelling_data[modelling_data[RESPONSE+"TrainingSet"]]
test_data = modelling_data[modelling_data[RESPONSE+"TestSet"]]
cal_data = modelling_data[modelling_data[RESPONSE+"ValidationSet"]]

In [8]:
X, y = modelling_data.drop(columns=[RESPONSE]), modelling_data[RESPONSE]
X_train, y_train = training_data.drop(columns=[RESPONSE]), training_data[RESPONSE]
X_test, y_test = test_data.drop(columns=[RESPONSE]), test_data[RESPONSE]
X_cal, y_cal = cal_data.drop(columns=[RESPONSE]), cal_data[RESPONSE]

In [9]:
X_preproc = X[FEATURES]
X_train_preproc = X_train[FEATURES]
X_test_preproc = X_test[FEATURES]
X_cal_preproc = X_cal[FEATURES]

In [10]:
X_train_preproc.head()

,type_Kick_a0,type_Handball_a0,quarter_a0,quarter_seconds_a0,overall_seconds_a0,left_right_start_x_a0,left_right_start_y_a0,left_right_end_x_a0,left_right_end_y_a0,dx_a0,dy_a0,movement_a0,type_Kick_a1,type_Handball_a1,quarter_a1,quarter_seconds_a1,overall_seconds_a1,left_right_start_x_a1,left_right_start_y_a1,left_right_end_x_a1,left_right_end_y_a1,dx_a1,dy_a1,movement_a1,type_Kick_a2,type_Handball_a2,quarter_a2,quarter_seconds_a2,overall_seconds_a2,left_right_start_x_a2,left_right_start_y_a2,left_right_end_x_a2,left_right_end_y_a2,dx_a2,dy_a2,movement_a2
1,False,True,1,29.0,29.0,-12.0,5.0,-22.0,2.0,-10.0,-3.0,10.440307,False,True,1,24.0,24.0,9.0,-6.0,12.0,-5.0,3.0,1.0,3.162278,False,True,1,24.0,24.0,9.0,-6.0,12.0,-5.0,3.0,1.0,3.162278
5,True,False,1,93.0,93.0,0.0,0.0,37.0,17.0,37.0,17.0,40.718546,True,False,1,39.0,39.0,26.0,-21.0,77.0,0.0,51.0,21.0,55.154329,False,True,1,37.0,37.0,11.0,-26.0,26.0,-21.0,15.0,5.0,15.811388
6,False,True,1,104.0,104.0,34.0,41.0,37.0,56.0,3.0,15.0,15.297059,True,False,1,93.0,93.0,0.0,0.0,37.0,17.0,37.0,17.0,40.718546,True,False,1,39.0,39.0,26.0,-21.0,77.0,0.0,51.0,21.0,55.154329
7,True,False,1,108.0,108.0,37.0,56.0,46.0,22.0,9.0,-34.0,35.171011,False,True,1,104.0,104.0,34.0,41.0,37.0,56.0,3.0,15.0,15.297059,True,False,1,93.0,93.0,0.0,0.0,37.0,17.0,37.0,17.0,40.718546
8,False,True,1,113.0,113.0,-42.0,-20.0,-46.0,-20.0,-4.0,0.0,4.000000,True,False,1,108.0,108.0,37.0,56.0,46.0,22.0,9.0,-34.0,35.171011,False,True,1,104.0,104.0,34.0,41.0,37.0,56.0,3.0,15.0,15.297059


Optuna Hyperparameter Tuning Class - HyperParameterTuner & XGBHyperparameterTuner

In [11]:
xgb_tuner = XGBHyperparameterTuner(X_train_preproc, y_train, monotonicity_constraints=MONOTONE_CONSTRAINTS)

In [12]:
xgb_tuner.tune_hyperparameters()

[I 2023-11-20 14:21:22,070] A new study created in memory with name: no-name-c470936a-c056-48e6-b429-1cb9f17d5a0d
[I 2023-11-20 14:21:23,174] Trial 0 finished with value: 0.26872712784596753 and parameters: {'max_depth': 4, 'min_child_weight': 15, 'eta': 0.19643348985470593, 'gamma': 1.7023573472475635, 'lambda': 3.9145101575003896, 'alpha': 0.00027955585834862646, 'subsample': 0.36618250448686085, 'colsample_bytree': 0.43529285878819685}. Best is trial 0 with value: 0.26872712784596753.
[I 2023-11-20 14:21:24,063] Trial 1 finished with value: 0.07462855602411635 and parameters: {'max_depth': 18, 'min_child_weight': 97, 'eta': 0.9678362264985308, 'gamma': 0.08355899459049466, 'lambda': 1.0653818701379143, 'alpha': 2.2485697919461636, 'subsample': 0.6081750341935399, 'colsample_bytree': 0.6491829188296261}. Best is trial 1 with value: 0.07462855602411635.
[I 2023-11-20 14:21:25,040] Trial 2 finished with value: 0.44553859415563496 and parameters: {'max_depth': 17, 'min_child_weight': 43

Number of finished trials:  100
Best trial:
  Value: 0.061309718668687244
  Params: 
    max_depth: 15
    min_child_weight: 9
    eta: 0.8323597736250923
    gamma: 0.387932465209438
    lambda: 1.158369267912131
    alpha: 6.041584764203979
    subsample: 0.6566786421300961
    colsample_bytree: 0.7541195347209855


In [13]:
params = xgb_tuner.get_best_params()
params

{'max_depth': 15,
 'min_child_weight': 9,
 'eta': 0.8323597736250923,
 'gamma': 0.387932465209438,
 'lambda': 1.158369267912131,
 'alpha': 6.041584764203979,
 'subsample': 0.6566786421300961,
 'colsample_bytree': 0.7541195347209855}

Training Model - SuperXGBClassifier class for training and predictions

In [14]:
params['objective'] = OptunaXGBParamGrid.error
params['num_rounds'] = 1000
params['early_stopping_rounds'] = 50
params['verbosity'] = 1
params['monotone_constraints'] = MONOTONE_CONSTRAINTS

In [15]:
super_xgb = SuperXGBClassifier(X_train = X_train_preproc, 
                               y_train = y_train, 
                               X_test = X_test_preproc, 
                               y_test = y_test,
                               X_cal = X_cal_preproc,
                               y_cal = y_cal,
                               params = params)

In [16]:
super_xgb.fit()

[0]	validation_0-logloss:0.20999	validation_1-logloss:0.21475
[1]	validation_0-logloss:0.13578	validation_1-logloss:0.14244
[2]	validation_0-logloss:0.09085	validation_1-logloss:0.09713
[3]	validation_0-logloss:0.07280	validation_1-logloss:0.07971
[4]	validation_0-logloss:0.06377	validation_1-logloss:0.07134
[5]	validation_0-logloss:0.05717	validation_1-logloss:0.06535
[6]	validation_0-logloss:0.05487	validation_1-logloss:0.06332
[7]	validation_0-logloss:0.05212	validation_1-logloss:0.06226
[8]	validation_0-logloss:0.05008	validation_1-logloss:0.06111
[9]	validation_0-logloss:0.04842	validation_1-logloss:0.06053
[10]	validation_0-logloss:0.04628	validation_1-logloss:0.05975
[11]	validation_0-logloss:0.04530	validation_1-logloss:0.05992
[12]	validation_0-logloss:0.04377	validation_1-logloss:0.05939
[13]	validation_0-logloss:0.04240	validation_1-logloss:0.05922
[14]	validation_0-logloss:0.04144	validation_1-logloss:0.05927
[15]	validation_0-logloss:0.04043	validation_1-logloss:0.05959
[1

In [17]:
super_xgb.xgb_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7541195347209855, device=None,
              early_stopping_rounds=50, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.387932465209438,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.8323597736250923,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=15, max_leaves=None,
              min_child_weight=9, missing=nan, monotone_constraints={},
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [18]:
super_xgb.xgb_model.get_booster().feature_names

['type_Kick_a0',
 'type_Handball_a0',
 'quarter_a0',
 'quarter_seconds_a0',
 'overall_seconds_a0',
 'left_right_start_x_a0',
 'left_right_start_y_a0',
 'left_right_end_x_a0',
 'left_right_end_y_a0',
 'dx_a0',
 'dy_a0',
 'movement_a0',
 'type_Kick_a1',
 'type_Handball_a1',
 'quarter_a1',
 'quarter_seconds_a1',
 'overall_seconds_a1',
 'left_right_start_x_a1',
 'left_right_start_y_a1',
 'left_right_end_x_a1',
 'left_right_end_y_a1',
 'dx_a1',
 'dy_a1',
 'movement_a1',
 'type_Kick_a2',
 'type_Handball_a2',
 'quarter_a2',
 'quarter_seconds_a2',
 'overall_seconds_a2',
 'left_right_start_x_a2',
 'left_right_start_y_a2',
 'left_right_end_x_a2',
 'left_right_end_y_a2',
 'dx_a2',
 'dy_a2',
 'movement_a2']

In [19]:
preds = super_xgb.predict(X_preproc)
train_preds = super_xgb.predict(X_train_preproc)
test_preds = super_xgb.predict(X_test_preproc)

In [20]:
probas = super_xgb.predict_proba(X_preproc)[:, 1]
train_probas = super_xgb.predict_proba(X_train_preproc)[:, 1]
test_probas = super_xgb.predict_proba(X_test_preproc)[:, 1]
cal_probas = super_xgb.predict_proba(X_cal_preproc)[:, 1]

In [21]:
super_xgb.calibrate()

In [22]:
cal_probas = super_xgb.predict_proba(X_preproc, calibrate=True)
train_cal_probas = super_xgb.predict_proba(X_train_preproc, calibrate=True)
test_cal_probas = super_xgb.predict_proba(X_test_preproc, calibrate=True)

Check Average Predictions

In [23]:
probas.mean(), modelling_data[RESPONSE].mean(), cal_probas.mean()

(0.71556765, 0.7152607375122261, 0.715066833461573)

In [24]:
train_probas.mean(), training_data[RESPONSE].mean(), train_cal_probas.mean()

(0.7163862, 0.7162080187145792, 0.7158845753253593)

In [25]:
test_probas.mean(), test_data[RESPONSE].mean(), test_cal_probas.mean()

(0.71330446, 0.7127536356226674, 0.7128191263561972)

Export model

In [26]:
super_xgb.export_model(model_output_path + "/" + model_file_name + ".joblib")

Export data and predictions

In [27]:
modelling_data['xgb_preds'] = preds
modelling_data['xgb_probas'] = probas
modelling_data['xgb_probas_cal'] = cal_probas
modelling_data.to_csv(prediction_output_path + '/predictions_' + model_file_name + '.csv', index = False)
modelling_data.head()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Season,action_type,Contested,Mark,Free,From_Centre_Bounce,Kick_Inside50,To_Ball_Up,From_Ball_Up,Rushed_Behind,Contest_Target,Goal,Behind,To_Out_On_Full,From_Out_On_Full,Error,From_Kick_In,end_x,end_y,pitch_start_x,pitch_start_y,pitch_end_x,pitch_end_y,left_right_start_x,left_right_start_y,left_right_end_x,left_right_end_y,start_distance_to_right_goal,end_distance_to_right_goal,Inside50,1,2,3,4,Duration,match_id,chain_number,order,quarter,quarter_seconds,overall_seconds,team,player,contested,mark,outcome_type,type_Kick_a0,type_Handball_a0,outcome_effective_a0,outcome_ineffective_a0,outcome_clanger_a0,type_Kick_outcome_effective_a0,type_Kick_outcome_ineffective_a0,type_Kick_outcome_clanger_a0,type_Handball_outcome_effective_a0,type_Handball_outcome_ineffective_a0,type_Handball_outcome_clanger_a0,quarter_a0,quarter_seconds_a0,overall_seconds_a0,left_right_start_x_a0,left_right_start_y_a0,left_right_end_x_a0,left_right_end_y_a0,dx_a0,dy_a0,movement_a0,contested_a0,mark_a0,type_Kick_a1,type_Handball_a1,outcome_effective_a1,outcome_ineffective_a1,outcome_clanger_a1,type_Kick_outcome_effective_a1,type_Kick_outcome_ineffective_a1,type_Kick_outcome_clanger_a1,type_Handball_outcome_effective_a1,type_Handball_outcome_ineffective_a1,type_Handball_outcome_clanger_a1,quarter_a1,quarter_seconds_a1,overall_seconds_a1,left_right_start_x_a1,left_right_start_y_a1,left_right_end_x_a1,left_right_end_y_a1,dx_a1,dy_a1,movement_a1,contested_a1,mark_a1,type_Kick_a2,type_Handball_a2,outcome_effective_a2,outcome_ineffective_a2,outcome_clanger_a2,type_Kick_outcome_effective_a2,type_Kick_outcome_ineffective_a2,type_Kick_outcome_clanger_a2,type_Handball_outcome_effective_a2,type_Handball_outcome_ineffective_a2,type_Handball_outcome_clanger_a2,quarter_a2,quarter_seconds_a2,overall_seconds_a2,left_right_start_x_a2,left_right_start_y_a2,left_right_end_x_a2,left_right_end_y_a2,dx_a2,dy_a2,movement_a2,contested_a2,mark_a2,team_1,team_2,time_delta1,time_delta2,dx_a01,dy_a01,move_a01,dx_a02,dy_a02,move_a02,goalscore_team,goalscore_opponent,goalscore_diff,Disposal_Response,Disposal_ResponseTrainingSet,Disposal_ResponseTestSet,Disposal_ResponseValidationSet,xgb_preds,xgb_probas,xgb_probas_cal
0,1,centreBounce,goal,3.0,1,13,24.0,Brisbane Lions,Brisbane Lions,Dayne Zorko,Dayne_Zorko,Handball,9.0,-6.0,ineffective,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,Handball,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,-5.0,9.0,-6.0,12.0,-5.0,9.0,-6.0,12.0,-5.0,69.260378,66.189123,NaN,1689.0,2119.0,2063.0,1920.0,24.0,202101_BrisbaneLions_Sydney,1,3.0,1,24.0,24.0,Brisbane Lions,Dayne Zorko,NaN,NaN,ineffective,False,True,False,True,False,False,False,False,False,True,False,1,24.0,24.0,9.0,-6.0,12.0,-5.0,3.0,1.0,3.162278,NaN,NaN,False,True,False,True,False,False,False,False,False,True,False,1,24.0,24.0,9.0,-6.0,12.0,-5.0,3.0,1.0,3.162278,NaN,NaN,False,True,False,True,False,False,False,False,False,True,False,1,24.0,24.0,9.0,-6.0,12.0,-5.0,3.0,1.0,3.162278,NaN,NaN,True,True,0.0,0.0,3.0,1.0,3.162278,3.0,1.0,3.162278,0,0,0,0,False,False,True,1,0.600806,0.588425
1,1,centreBounce,goal,5.0,1,13,29.0,Brisbane Lions,Sydney,Oliver Florent,Oliver_Florent,Handball,12.0,-5.0,effective,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,Handball,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,-2.0,12.0,-5.0,22.0,-2.0,-12.0,5.0,-22.0,2.0,90.138782,100.019998,NaN,1689.0,2119.0,2063.0,1920.0,29.0,202101_BrisbaneLions_Sydney,1,5.0,1,29.0,29.0,Sydney,Oliver Florent,NaN,NaN,effective,False,True,True,False,False,False,False,False,True,False,False,1,29.0,29.0,-12.0,5.0,-22.0,2.0,-10.0,-3.0,10.440307,Na